In [1]:
import streamlit as st
import ast
import re
from collections import deque
from typing import Optional, List, Dict


In [12]:
def parse_final_states(s: str) -> set:
    s = s.strip().strip("{}")
    return {x.strip().strip("'\"") for x in s.split(",") if x.strip()}

def safe_eval_transitions(s: str):
    s = s.strip().replace("'", '"')
    s = re.sub(r"\b([a-zA-Z0-9_]+)\b(?=\s*:|\s*,\s*[\{\}\)])", r'"\1"', s)
    s = re.sub(r"\(\s*([a-zA-Z0-9_]+)\s*,", r'("\1",', s)
    s = re.sub(r",\s*([a-zA-Z0-9_]+)\s*\)", r', "\1")', s)
    return ast.literal_eval(s)


In [25]:
from collections import deque
from typing import Optional, List, Dict, Tuple, Any
import ast
import re

def parse_final_states(s: str) -> set:
    s = s.strip().strip("{}")
    return {x.strip().strip("'\"") for x in s.split(",") if x.strip()}

def safe_eval_transitions(s: str):
    s = s.strip().replace("'", '"')
    s = re.sub(r"\b([a-zA-Z0-9_]+)\b(?=\s*:|\s*,\s*[\{\}\)])", r'"\1"', s)
    s = re.sub(r"\(\s*([a-zA-Z0-9_]+)\s*,", r'("\1",', s)
    s = re.sub(r",\s*([a-zA-Z0-9_]+)\s*\)", r', "\1")', s)
    return ast.literal_eval(s)

class NPDA:
    def __init__(self, delta, start_state: str, start_stack: str, final_states: set):
        self.delta = delta
        self.q0 = start_state
        self.z0 = start_stack
        self.F = final_states

    def get_success_path(self, input_string: str) -> Optional[List[Dict[str, Any]]]:
        queue = deque()
        # (state, remaining_input, stack_list, path_so_far)
        initial_stack = [self.z0]
        initial_config = (self.q0, input_string, initial_stack, [])
        queue.append(initial_config)
        
        # Initial step (starting configuration)
        initial_step = {
            "state": self.q0,
            "input_left": input_string,
            "stack": initial_stack.copy(),
            "description": "START"
        }
        # path will contain full steps including descriptions
        queue.append((self.q0, input_string, initial_stack, [initial_step]))
        
        visited = set()
        i =0
        while queue:
            state, inp, stack, path = queue.popleft()
            i+=1
            print(f"Step {i}: State={state}, Input='{inp}', Stack={stack}, Path length={len(path)}")
            # Create current configuration snapshot
            current = {
                "state": state,
                "input_left": inp,
                "stack": stack.copy(),
                "description": ""  # will be filled by the move that led here
            }

            # Acceptance condition: no input left and in final state
            if not inp and state in self.F:
                # Mark the final step as accepted
                final_step = {**current, "description": "ACCEPTED"}
                return path + [final_step]

            # Avoid revisiting the exact same configuration
            config_key = (state, inp, tuple(stack))
            if config_key in visited:
                continue
            visited.add(config_key)

            # Must have something on stack to pop
            if not stack:
                continue
            top = stack[-1]

            moves = []

            # 1. Consume input symbol (if available)
            if inp:
                key = (state, inp[0], top)
                if key in self.delta:
                    for new_state, push in self.delta[key]:
                        symbol_read = inp[0]
                        if push == "":
                            desc = f"Read '{symbol_read}' → pop '{top}'"
                        else:
                            desc = f"Read '{symbol_read}' → pop '{top}', push '{push}'"
                        moves.append((new_state, inp[1:], push, desc))

            # 2. ε-transition (no input consumed)
            key = (state, "", top)
            if key in self.delta:
                for new_state, push in self.delta[key]:
                    if push == "":
                        desc = f"ε-move → pop '{top}'"
                    else:
                        desc = f"ε-move → pop '{top}', push '{push}'"
                    moves.append((new_state, inp, push, desc))

            # Apply all possible moves
            for new_state, new_inp, push, desc in moves:
                new_stack = stack[:-1]  # pop top
                if push:  # push can be string like "((" or "Z" or ""
                    # Push in reverse order (rightmost symbol goes on top)
                    new_stack.extend(reversed(push))

                # Build new path: previous path + current state with description of the move
                new_path_entry = {**current, "description": desc}
                new_path = path + [new_path_entry]

                queue.append((new_state, new_inp, new_stack, new_path))

        return None  # No accepting path found

In [ ]:
presets = {
        "Balanced Parentheses": {"start":"q0","stack":"Z","final":"{q1}","trans":'{("q0","(","Z"):[("q0","(Z")],("q0","(","("):[("q0","((")],("q0",")","("):[("q0","")],("q0","","Z"):[("q1","Z")]}',"input":"(())(())"},
    }

In [27]:
start = presets["Balanced Parentheses"]["start"]
stack = presets["Balanced Parentheses"]["stack"]
final = presets["Balanced Parentheses"]["final"]
trans = presets["Balanced Parentheses"]["trans"]
inp   = presets["Balanced Parentheses"]["input"]
print(start, stack, final, inp)

q0 Z {q1} (()())


In [ ]:
'{("q0","(","Z"):[("q0","(Z")],
("q0","(","("):[("q0","((")],
("q0",")","("):[("q0","")],
("q0","","Z"):[("q1","Z")]}'

In [28]:
delta = safe_eval_transitions(trans)
finals = parse_final_states(final)
npda = NPDA(delta, start, stack, finals)
trace = npda.get_success_path(inp)

Step 1: State=q0, Input='(()())', Stack=['Z'], Path length=0
Step 2: State=q0, Input='(()())', Stack=['Z'], Path length=1
Step 3: State=q0, Input='()())', Stack=['Z', '('], Path length=1
Step 4: State=q1, Input='(()())', Stack=['Z'], Path length=1
Step 5: State=q0, Input=')())', Stack=['Z', '(', '('], Path length=2
Step 6: State=q0, Input='())', Stack=['Z', '('], Path length=3
Step 7: State=q0, Input='))', Stack=['Z', '(', '('], Path length=4
Step 8: State=q0, Input=')', Stack=['Z', '('], Path length=5
Step 9: State=q0, Input='', Stack=['Z'], Path length=6
Step 10: State=q1, Input='', Stack=['Z'], Path length=7


In [24]:
path = trace
if path:
    for step in path:
        print(step)

{'state': 'q0', 'input_left': '(()())', 'stack': ['Z'], 'description': "Read '(' → pop 'Z', push '(Z'"}
{'state': 'q0', 'input_left': '()())', 'stack': ['Z', '('], 'description': "Read '(' → pop '(', push '(('"}
{'state': 'q0', 'input_left': ')())', 'stack': ['Z', '(', '('], 'description': "Read ')' → pop '('"}
{'state': 'q0', 'input_left': '())', 'stack': ['Z', '('], 'description': "Read '(' → pop '(', push '(('"}
{'state': 'q0', 'input_left': '))', 'stack': ['Z', '(', '('], 'description': "Read ')' → pop '('"}
{'state': 'q0', 'input_left': ')', 'stack': ['Z', '('], 'description': "Read ')' → pop '('"}
{'state': 'q0', 'input_left': '', 'stack': ['Z'], 'description': "ε-move → pop 'Z', push 'Z'"}
{'state': 'q1', 'input_left': '', 'stack': ['Z'], 'description': 'ACCEPTED'}
